In [2]:
import pandas as pd
import numpy as np
import requests
import datetime
import datetime as dt
import calendar
import matplotlib.pyplot as plt

In [3]:
#This function downloads the UK electricity generation mix for a given period
def data_retrieval(date):
    headers = {
  'Accept': 'application/json'
      }
    raw_url='https://api.carbonintensity.org.uk/generation/'

    # Load the selected dataset for a given period. Τhe datasets are loaded as json objects. 
    #Each json object contains the electricity generation share (as a percentage) for each 30 min interval
    r = requests.get(raw_url+date, params={}, headers = headers)

    raw_data=r.json() 
    
    #The starting periods of each object are loaded as a list
    dates=list()
    for i in range(len(raw_data['data'])):
        dates.append(raw_data['data'][i]['from'])
    
    #The share of each source is loaded as a list 
    biomass=list()
    for i in range(len(raw_data['data'])):
        biomass.append(raw_data['data'][i]['generationmix'][0]['perc'])
    
    coal=list()
    for i in range(len(raw_data['data'])):
        coal.append(raw_data['data'][i]['generationmix'][1]['perc'])

    imports=list()
    for i in range(len(raw_data['data'])):
        imports.append(raw_data['data'][i]['generationmix'][2]['perc'])
    
    gas=list()
    for i in range(len(raw_data['data'])):
        gas.append(raw_data['data'][i]['generationmix'][3]['perc'])

    nuclear=list()
    for i in range(len(raw_data['data'])):
        nuclear.append(raw_data['data'][i]['generationmix'][4]['perc'])   
    
    other=list()
    for i in range(len(raw_data['data'])):
        other.append(raw_data['data'][i]['generationmix'][5]['perc'])  

    hydro=list()
    for i in range(len(raw_data['data'])):
        hydro.append(raw_data['data'][i]['generationmix'][6]['perc']) 

    solar=list()
    for i in range(len(raw_data['data'])):
        solar.append(raw_data['data'][i]['generationmix'][7]['perc']) 
    
    wind=list()
    for i in range(len(raw_data['data'])):
        wind.append(raw_data['data'][i]['generationmix'][8]['perc']) 

    #A data frame is created and columnx contain the share of electricity sources.
    df=pd.DataFrame(dates)
    df['dates']=df[0]
    df['gas']=gas
    df['coal']=coal
    df['biomass']=biomass
    df['hydro']=hydro
    df['solar']=solar
    df['wind']=wind
    df['imports']=imports
    df['nuclear']=nuclear
    df['other']=other
    df=df.drop(0,axis=1)

    return df

#The periods for each month are created
str1=[]
str2=[]
month=[]

#The date format is suitably changed by using the datetime library to match the website's format
for i in range(12):
    str1.append(datetime.datetime.strptime(str(dt.datetime(2019, i+1, 1, 0, 1)), "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%dT%H:%MZ/'))
    if i<11:
        str2.append(datetime.datetime.strptime(str(dt.datetime(2019, i+2, 1, 0, 0)), "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%dT%H:%MZ'))
    else:
        str2.append(datetime.datetime.strptime(str(dt.datetime(2020, 1, 1, 0, 0)), "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%dT%H:%MZ'))

    month.append(str1[i]+str2[i])

In [4]:
#The generation mix of each month is stored in different CSV files for later use.
for i in range(12):
    data_retrieval(month[i]).to_csv(r'Gen_Mix_Month_%s.csv' %(i+1), index = False)

In [5]:
#Data is loaded from the CSVs 
for i in range(12):
    globals()['Gen_Mix_Month_%s' %(i+1)] = pd.read_csv('Gen_Mix_Month_%s.csv' %(i+1))
    
#Create a new set of dataframes by grouping the generation sources
for i in range(12):
     globals()['Sum_Gen_Mix_Month_%s' %(i+1)] = pd.DataFrame([])

for i in range(12):
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Date'] = globals()['Gen_Mix_Month_%s' %(i+1)]['dates']
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Renewables'] = globals()['Gen_Mix_Month_%s' %(i+1)]['biomass'] + globals()['Gen_Mix_Month_%s' %(i+1)]['hydro'] + globals()['Gen_Mix_Month_%s' %(i+1)]['solar'] + globals()['Gen_Mix_Month_%s' %(i+1)]['wind']
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Fossil_Fuels'] = globals()['Gen_Mix_Month_%s' %(i+1)]['gas'] + globals()['Gen_Mix_Month_%s' %(i+1)]['coal']
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Imports'] = globals()['Gen_Mix_Month_%s' %(i+1)]['imports']
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Nuclear'] = globals()['Gen_Mix_Month_%s' %(i+1)]['nuclear']
    globals()['Sum_Gen_Mix_Month_%s' %(i+1)]['Other'] = globals()['Gen_Mix_Month_%s' %(i+1)]['other']

#Sum up the datasets from all months and extract the descriptive statistics
selected_months=[]
for i in range(12):
    selected_months.append(globals()['Sum_Gen_Mix_Month_%s' %(i+1)])
sum_all_data= pd.concat(selected_months, axis=0)
sum_all_data.reset_index(drop=True, inplace=True)

In [6]:
sum_all_data.to_csv('2019_GenMix.csv')